In [ ]:
from urllib.request import urlretrieve
from PIL import Image
import os

import pandas as pd
import shutil
data_dir = '/content/diffusiondb-part-1-to-5'

In [ ]:

part_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/images/part-000001.zip'
urlretrieve(part_url, f'part-000001.zip')

part_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/images/part-000002.zip'
urlretrieve(part_url, f'part-000002.zip')

part_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/images/part-000003.zip'
urlretrieve(part_url, f'part-000003.zip')

part_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/images/part-000004.zip'
urlretrieve(part_url, f'part-000004.zip')

part_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/images/part-000005.zip'
urlretrieve(part_url, f'part-000005.zip')

('part-000005.zip', <http.client.HTTPMessage at 0x78255ea12200>)

In [ ]:
shutil.unpack_archive(f'/content/part-000001.zip', f'diffusiondb-part-1-to-5')
shutil.unpack_archive(f'/content/part-000002.zip', f'diffusiondb-part-1-to-5')
shutil.unpack_archive(f'/content/part-000003.zip', f'diffusiondb-part-1-to-5')
shutil.unpack_archive(f'/content/part-000004.zip', f'diffusiondb-part-1-to-5')
shutil.unpack_archive(f'/content/part-000005.zip', f'diffusiondb-part-1-to-5')

In [ ]:
os.remove(f'{data_dir}/part-000001.json')
os.remove(f'{data_dir}/part-000002.json')
os.remove(f'{data_dir}/part-000003.json')
os.remove(f'{data_dir}/part-000004.json')
os.remove(f'{data_dir}/part-000005.json')
len(os.listdir(data_dir))

5000

In [ ]:
metadata = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(metadata, f'metadata.parquet')

('metadata.parquet', <http.client.HTTPMessage at 0x7f566f162bc0>)

In [ ]:
metadata_df = pd.read_parquet('metadata.parquet')

In [ ]:
metadata_df[:5000]

In [ ]:
metadata_df = metadata_df[:5000]
nsfw_filtered = metadata_df.loc[(metadata_df['image_nsfw'] <= .6) & (
    metadata_df['prompt_nsfw'] <= .2)]
nsfw_filtered

### Rate image using aesthetic-scorer


In [ ]:
!git clone https://github.com/kenjiqq/aesthetics-scorer

Cloning into 'aesthetics-scorer'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 309 (delta 12), reused 21 (delta 5), pack-reused 263
Receiving objects: 100% (309/309), 373.88 MiB | 34.70 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Updating files: 100% (61/61), done.


In [ ]:
import torch
import torch.nn as nn
import json
import os


class AestheticScorer(nn.Module):
    def __init__(self, input_size=0, use_activation=False, dropout=0.2, config=None, hidden_dim=1024, reduce_dims=False, output_activation=None):
        super().__init__()
        self.config = {
            "input_size": input_size,
            "use_activation": use_activation,
            "dropout": dropout,
            "hidden_dim": hidden_dim,
            "reduce_dims": reduce_dims,
            "output_activation": output_activation
        }
        if config != None:
            self.config.update(config)

        layers = [
            nn.Linear(self.config["input_size"], self.config["hidden_dim"]),
            nn.ReLU() if self.config["use_activation"] else None,
            nn.Dropout(self.config["dropout"]),

            nn.Linear(self.config["hidden_dim"], round(
                self.config["hidden_dim"] / (2 if reduce_dims else 1))),
            nn.ReLU() if self.config["use_activation"] else None,
            nn.Dropout(self.config["dropout"]),

            nn.Linear(round(self.config["hidden_dim"] / (2 if reduce_dims else 1)), round(
                self.config["hidden_dim"] / (4 if reduce_dims else 1))),
            nn.ReLU() if self.config["use_activation"] else None,
            nn.Dropout(self.config["dropout"]),

            nn.Linear(round(self.config["hidden_dim"] / (4 if reduce_dims else 1)), round(
                self.config["hidden_dim"] / (8 if reduce_dims else 1))),
            nn.ReLU() if self.config["use_activation"] else None,
            nn.Linear(
                round(self.config["hidden_dim"] / (8 if reduce_dims else 1)), 1),
        ]
        if self.config["output_activation"] == "sigmoid":
            layers.append(
                nn.Sigmoid()
            )
        layers = [x for x in layers if x is not None]
        self.layers = nn.Sequential(
            *layers
        )

    def forward(self, x):
        if self.config["output_activation"] == "sigmoid":
            upper, lower = 10, 1
            scale = upper - lower
            return (self.layers(x) * scale) + lower
        else:
            return self.layers(x)

    def save(self, save_name):
        split_name = os.path.splitext(save_name)
        with open(f"{split_name[0]}.config", "w") as outfile:
            outfile.write(json.dumps(self.config, indent=4))

        for i in range(6):  # saving sometiles fails, so retry 5 times, might be windows issue
            try:
                torch.save(self.state_dict(), save_name)
                break
            except RuntimeError as e:
                # check if error contains string "File"
                if "cannot be opened" in str(e) and i < 5:
                    print("Model save failed, retrying...")
                else:
                    raise e


def preprocess(embeddings):
    return embeddings / embeddings.norm(p=2, dim=-1, keepdim=True)


def load_model(weight_path, device='cuda' if torch.cuda.is_available() else 'cpu'):
    split_path = os.path.splitext(weight_path)
    with open(f"{split_path[0]}.config", "r") as config_file:
        config = json.load(config_file)
    model = AestheticScorer(config=config)
    model.load_state_dict(torch.load(weight_path, map_location=device))
    model.eval()
    return model

In [ ]:
from transformers import CLIPModel, CLIPProcessor

MODEL = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CLIPModel.from_pretrained(MODEL)
vision_model = model.vision_model
vision_model.to(DEVICE)
del model
clip_processor = CLIPProcessor.from_pretrained(MODEL)

rating_model = load_model(
    "/content/aesthetics-scorer/aesthetics_scorer/models/aesthetics_scorer_rating_openclip_vit_h_14.pth").to(DEVICE)
artifacts_model = load_model(
    "/content/aesthetics-scorer/aesthetics_scorer/models/aesthetics_scorer_artifacts_openclip_vit_h_14.pth").to(DEVICE)


def predict(img):
    inputs = clip_processor(images=img, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        vision_output = vision_model(**inputs)
    pooled_output = vision_output.pooler_output
    embedding = preprocess(pooled_output)
    with torch.no_grad():
        rating = rating_model(embedding)
        artifact = artifacts_model(embedding)
    return rating.detach().cpu().item(), artifact.detach().cpu().item()

In [ ]:
# def predict(img):
#     inputs = clip_processor(images=img, return_tensors="pt").to(DEVICE)
#     with torch.no_grad():
#         vision_output = vision_model(**inputs)
#     pooled_output = vision_output.pooler_output
#     embedding = preprocess(pooled_output)
#     with torch.no_grad():
#         rating = rating_model(embedding)
#         artifact = artifacts_model(embedding)
#     return lambda x :x[0] ,rating.detach().cpu(), artifact.detach().cpu().tolist()

In [ ]:
# def rateImage(image_names):
#   images = []
#   for image_name in image_names:
#     images.append(Image.open(f'{data_dir}/{image_name}'))
#   return predict(images)

In [ ]:
# rateImage(['058d0e39-92f3-403c-8b64-8411b5062e82.png','92c488a0-4297-45c1-a001-76aee6399d30.png','ba37e532-41f6-46ef-a521-1dff30cfbce5.png','bcaacfc0-36ed-4de7-8ff7-1bf1f2912b79.png',])

In [ ]:
# test = metadata_df.loc[(metadata_df['image_nsfw'] <= .6) & (metadata_df['prompt_nsfw'] <= .2)]

In [ ]:
# test[:1000]['rating'], test[:1000]['artifact'] = rateImage(test[:1000]['image_name'])

In [ ]:
def rateImage(image_names):
    ratings = []
    artifacts = []
    for image_name in image_names:
        rating, artifact = predict(Image.open(f'{data_dir}/{image_name}'))
        ratings.append(rating)
        artifacts.append(artifact)
    return ratings, artifacts

In [ ]:
nsfw_filtered['rating'], nsfw_filtered['artifact'] = rateImage(
    nsfw_filtered['image_name'])
nsfw_filtered

In [ ]:
nsfw_filtered.to_parquet(path='/content/Prompts-search/nsfw_filtered.parquet')

In [ ]:
nsfw_filtered = pd.read_parquet(
    '/content/Prompts-search/nsfw_filtered.parquet')

In [ ]:
aesthetics_filtered = nsfw_filtered.loc[(
    nsfw_filtered['rating'] >= 6.5) & (nsfw_filtered['artifact'] <= 2)]
aesthetics_filtered.drop_duplicates(subset=['image_name'])

In [ ]:
Image.open(f'{data_dir}/dc71658a-5e4b-4dca-861a-e1535510348b.png')

### Add a new pandas column contain base64 encoded string of images


In [ ]:
import base64


def addImageBase64Column(row):
    with open(f"{data_dir}/{row['image_name']}", "rb") as image_file:
        return base64.b64encode(image_file.read()).decode()


aesthetics_filtered['image'] = aesthetics_filtered.apply(
    addImageBase64Column, axis=1)

In [ ]:
dropped = aesthetics_filtered.drop(['user_name', 'timestamp', 'image_nsfw',
                                   'prompt_nsfw', 'part_id', 'width', 'height', 'rating', 'artifact'], axis=1)
dropped['id'] = dropped['image_name'].str[:-4]
dropped

### Export to JSON


In [ ]:
with open(f'/content/diffusiondb-part-1-to-5-with-base64-images-full.jsonl', 'w') as f:
    json_full = dropped.to_json(orient='records', lines=True)
    f.write(json_full)

In [ ]:
with open(f'/content/20-images.jsonl', 'w') as f:
    test_json = dropped[:20].to_json(orient='records', lines=True)
    f.write(test_json)

### Create a zip file of filtered images


In [1]:
import shutil

In [ ]:
for image_name in dropped['id']:
    shutil.copy2(f"{data_dir}/{image_name}",
                 f'/content/diffusiondb-images/{image_name}')
len(os.listdir('/content/diffusiondb-images'))

In [ ]:
shutil.make_archive('/content/Prompts-search/diffusiondb-images',
                    'zip', '/content/diffusiondb-images')